In [8]:
import sys
from tqdm import tqdm

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *
from calculus_path_mod.terms import num_comparison, nat_numbers, arithm_ops, combinators, pairs, logic

from calculus_path_mod.terms.pseudonym import *

In [9]:
REDEX_TYPE = {
    "none": "none",
    "norm": "NORM-REDEX",
    "reduce": "REDUCE-REDEX",
    "mult": "MULT-REDEX"
}


class RedexesColoring:
    def _get_redexes_indexes(self, term: Term, init_index: int = 0, path_depth: int = 0) -> dict:
        """
        Using Left Outer strategy find all redexes, it's coordinates in the term tree,
        and the depth value for each redex.

        :param term: some Term is a current vertex
        :param init_index: some index is a coordinate of the current vertex in the term tree
        :param path_depth: depth in the term tree (simply recursion layer).
        :return: dict where a key is a index (coordinate of redex in the term tree),
                 and var is a depth from the tree root
        """

        if (term.kind == "atom") or (len(term.redexes) == 0):
            raise ValueError("The term doesn't contain a redex")
        if term.kind == "application":
            result = dict()
            if term.is_beta_redex:
                result = {**result,
                          **{init_index + 1: path_depth + 1}}
            if len(term._data[0].redexes) != 0:
                result = {**result,
                          **self._get_redexes_indexes(term._data[0], init_index + 1, path_depth + 1)}
            if len(term._data[1].redexes) != 0:
                result = {**result,
                          **self._get_redexes_indexes(term._data[1],
                                                      init_index + term._data[0].vertices_number + 1,
                                                      path_depth + 1)}
            return result
            # self is Abstraction:
        return self._get_redexes_indexes(term._data[1], init_index + 1, path_depth + 1)

    def get_colored_redexes(self, term: Term) -> dict:
        dict_redexes_indexes = self._get_redexes_indexes(term)
        dict_result = dict()
        for key_, item_ in dict_redexes_indexes.items():
            dict_result[key_] = [item_, REDEX_TYPE["none"]]

        for index_ in dict_result.keys():
            redex_term = term.subterm(index_)

            obj_term = redex_term._data[0]
            subj_term = redex_term._data[1]

            # look at obj-body
            obj_x_var = obj_term._data[0]._data
            obj_body_vars = obj_term._data[1]._vars

            # if obj-body has free x
            if obj_x_var in obj_body_vars.keys():  # check if x in obj_body
                if obj_body_vars[obj_x_var]['free'] == 0:  # x is NOT free in obj_body
                    if len(subj_term.redexes) == 0:
                        dict_result[index_][1] = REDEX_TYPE["norm"]
                    else:
                        dict_result[index_][1] = REDEX_TYPE["reduce"]
                elif obj_body_vars[obj_x_var]['free'] == 1:  # x is free in obj_body
                    if subj_term.kind == "atom":
                        dict_result[index_][1] = REDEX_TYPE["norm"]
                    elif subj_term.kind == "application":
                        dict_result[index_][1] = REDEX_TYPE["norm"]
                    else:  # TODO find out the way to say it more precisely
                        if obj_term._data[1].kind == 'atom':
                            dict_result[index_][1] = REDEX_TYPE["norm"]
                        else:
                            dict_result[index_][1] = REDEX_TYPE["mult"]
                else:  # many x is free in obj_body
                    if subj_term.kind == "atom":
                        dict_result[index_][1] = REDEX_TYPE["norm"]
                    elif subj_term.kind == "application":
                        if len(subj_term.redexes) == 0:
                            dict_result[index_][1] = REDEX_TYPE["norm"]
                        else:
                            dict_result[index_][1] = REDEX_TYPE["mult"]
                    else:  # TODO find out the way to say it more precisely
                        dict_result[index_][1] = REDEX_TYPE["mult"]
            else:  # x is not in obj_body
                if len(subj_term.redexes) == 0:
                    dict_result[index_][1] = REDEX_TYPE["norm"]
                else:
                    dict_result[index_][1] = REDEX_TYPE["reduce"]

            print("\n\n------------------>")
            print(f"index {index_}, term: {redex_term.funky_str()}")
            print(f"obj_term: {obj_term.funky_str()}")
            print(f"body obj_term vars: {obj_term._data[1]._vars}")
            print(f"subj_term: {subj_term.funky_str()}")

        return dict_result

# Testing term

In [10]:
def i_term():
    x = Var()
    x_ = Atom(x)
    return Lambda(x, x_)


def omega_term(n: int):
    x = Var()
    x_ = Atom(x)
    body_app_term = x_
    for _ in range(n - 1):
        body_app_term = App(body_app_term, x_)
    return Lambda(x, body_app_term)


def multi_i_term(m: int):
    app_term = i_term()
    for _ in range(m - 1):
        app_term = App(app_term, i_term())
    return app_term


def omega_multi_i_term(n: int, m: int):
    return App(omega_term(n), multi_i_term(m))

In [11]:
testing_term = omega_multi_i_term(3, 3)
testing_term.funky_str()

'((λx.((x x) x)) (((λy.y) (λa.a)) (λb.b)))'

# Testing coloring

In [12]:
rc = RedexesColoring()
print(rc.get_colored_redexes(testing_term))
print("\n\n***********************\nUPDATE BOUND VARS")
print(rc.get_colored_redexes(testing_term._update_bound_vars()))



------------------>
index 1, term: ((λx.((x x) x)) (((λy.y) (λa.a)) (λb.b)))
obj_term: (λx.((x x) x))
body obj_term vars: {15: {'free': 3, 'bound': 0}}
subj_term: (((λx.x) (λy.y)) (λa.a))


------------------>
index 9, term: ((λx.x) (λy.y))
obj_term: (λx.x)
body obj_term vars: {16: {'free': 1, 'bound': 0}}
subj_term: (λx.x)
{1: [1, 'MULT-REDEX'], 9: [3, 'NORM-REDEX']}


***********************
UPDATE BOUND VARS


------------------>
index 1, term: ((λx.((x x) x)) (((λy.y) (λa.a)) (λb.b)))
obj_term: (λx.((x x) x))
body obj_term vars: {19: {'free': 3, 'bound': 0}}
subj_term: (((λx.x) (λy.y)) (λa.a))


------------------>
index 9, term: ((λx.x) (λy.y))
obj_term: (λx.x)
body obj_term vars: {20: {'free': 1, 'bound': 0}}
subj_term: (λx.x)
{1: [1, 'MULT-REDEX'], 9: [3, 'NORM-REDEX']}


# Other terms for testing

In [13]:
def testing_procedure(t_term: Term):
    rc_ = RedexesColoring()

    print(f"term: {t_term.funky_str()}")
    # print(f"term: {t_term}")
    print("\n\n", rc_.get_colored_redexes(t_term))

    print("\n\n<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<\nUPDATE BOUND VARS")
    t_term = t_term._update_bound_vars()
    print(f"term: {t_term.funky_str()}")
    # print(f"term: {t_term}")
    print("\n\n", rc_.get_colored_redexes(t_term))

## ** REDUCE TERM **********************************************************************

In [14]:
def reduce_term_1():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, App(y_, Lambda(x, x_)))
    subj_term = App(
        App(Lambda(x, App(x_, x_)), Lambda(y, Lambda(z, multi_app_term(y_, z_, x_)))),
        App(Lambda(z, App(z_, z_)), Lambda(y, Lambda(z, multi_app_term(y_, z_, x_))))
    )

    return App(obj_term, subj_term)


testing_procedure(reduce_term_1())

term: ((λx.(y (λx.x))) (((λx.(x x)) (λy.(λa.((y a) x)))) ((λa.(a a)) (λy.(λa.((y a) x))))))


------------------>
index 1, term: ((λx.(y (λx.x))) (((λx.(x x)) (λy.(λa.((y a) x)))) ((λa.(a a)) (λy.(λa.((y a) x))))))
obj_term: (λx.(y (λx.x)))
body obj_term vars: {24: {'free': 1, 'bound': 0}, 23: {'free': 0, 'bound': 1}}
subj_term: (((λx.(x x)) (λy.(λa.((y a) x)))) ((λa.(a a)) (λy.(λa.((y a) x)))))


------------------>
index 8, term: ((λx.(x x)) (λy.(λa.((y a) x))))
obj_term: (λx.(x x))
body obj_term vars: {23: {'free': 2, 'bound': 0}}
subj_term: (λx.(λy.((x y) a)))


------------------>
index 20, term: ((λx.(x x)) (λy.(λx.((y x) a))))
obj_term: (λx.(x x))
body obj_term vars: {25: {'free': 2, 'bound': 0}}
subj_term: (λx.(λy.((x y) a)))


 {1: [1, 'REDUCE-REDEX'], 8: [3, 'MULT-REDEX'], 20: [3, 'MULT-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.(y (λa.a))) (((λb.(b b)) (λc.(λd.((c d) e)))) ((λj.(j j)) (λi.(λn.((i n) e))))))


------------------>
index 1, term

## * NORM TERM ***************************************************************************

In [15]:
def norm_term_1():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, App(y_, Lambda(x, x_)))
    subj_term = Lambda(x, x_)

    return App(obj_term, subj_term)


testing_procedure(norm_term_1())

term: ((λx.(y (λx.x))) (λx.x))


------------------>
index 1, term: ((λx.(y (λx.x))) (λx.x))
obj_term: (λx.(y (λx.x)))
body obj_term vars: {36: {'free': 1, 'bound': 0}, 35: {'free': 0, 'bound': 1}}
subj_term: (λx.x)


 {1: [1, 'NORM-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.(y (λa.a))) (λb.b))


------------------>
index 1, term: ((λx.(y (λa.a))) (λb.b))
obj_term: (λx.(y (λa.a)))
body obj_term vars: {36: {'free': 1, 'bound': 0}, 40: {'free': 0, 'bound': 1}}
subj_term: (λx.x)


 {1: [1, 'NORM-REDEX']}


In [19]:
def norm_term_2_v0():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, App(x_, Lambda(y, y_)))
    subj_term = z_

    return App(obj_term, subj_term)


testing_procedure(norm_term_2_v0())

term: ((λx.(x (λy.y))) a)


------------------>
index 1, term: ((λx.(x (λy.y))) a)
obj_term: (λx.(x (λy.y)))
body obj_term vars: {58: {'free': 1, 'bound': 0}, 59: {'free': 0, 'bound': 1}}
subj_term: x


 {1: [1, 'NORM-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.(x (λy.y))) a)


------------------>
index 1, term: ((λx.(x (λy.y))) a)
obj_term: (λx.(x (λy.y)))
body obj_term vars: {61: {'free': 1, 'bound': 0}, 62: {'free': 0, 'bound': 1}}
subj_term: x


 {1: [1, 'NORM-REDEX']}


In [21]:
def norm_term_2_v1():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, App(x_, Lambda(y, y_)))
    subj_term = multi_app_term(z_, z_, z_, z_)

    return App(obj_term, subj_term)


testing_procedure(norm_term_2_v1())

term: ((λx.(x (λy.y))) (((a a) a) a))


------------------>
index 1, term: ((λx.(x (λy.y))) (((a a) a) a))
obj_term: (λx.(x (λy.y)))
body obj_term vars: {68: {'free': 1, 'bound': 0}, 69: {'free': 0, 'bound': 1}}
subj_term: (((x x) x) x)


 {1: [1, 'NORM-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.(x (λy.y))) (((a a) a) a))


------------------>
index 1, term: ((λx.(x (λy.y))) (((a a) a) a))
obj_term: (λx.(x (λy.y)))
body obj_term vars: {71: {'free': 1, 'bound': 0}, 72: {'free': 0, 'bound': 1}}
subj_term: (((x x) x) x)


 {1: [1, 'NORM-REDEX']}


In [23]:
def norm_term_2_v2():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, multi_app_term(z_, z_, x_))
    subj_term = y_

    return App(obj_term, subj_term)


testing_procedure(norm_term_2_v2())

term: ((λx.((y y) x)) a)


------------------>
index 1, term: ((λx.((y y) x)) a)
obj_term: (λx.((y y) x))
body obj_term vars: {78: {'free': 2, 'bound': 0}, 76: {'free': 1, 'bound': 0}}
subj_term: x


 {1: [1, 'NORM-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.((y y) x)) a)


------------------>
index 1, term: ((λx.((y y) x)) a)
obj_term: (λx.((y y) x))
body obj_term vars: {78: {'free': 2, 'bound': 0}, 79: {'free': 1, 'bound': 0}}
subj_term: x


 {1: [1, 'NORM-REDEX']}


In [24]:
def norm_term_2_v3():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, multi_app_term(z_, z_, x_))
    subj_term = multi_app_term(y_, y_, y_, y_)

    return App(obj_term, subj_term)


testing_procedure(norm_term_2_v3())

term: ((λx.((y y) x)) (((a a) a) a))


------------------>
index 1, term: ((λx.((y y) x)) (((a a) a) a))
obj_term: (λx.((y y) x))
body obj_term vars: {82: {'free': 2, 'bound': 0}, 80: {'free': 1, 'bound': 0}}
subj_term: (((x x) x) x)


 {1: [1, 'NORM-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.((y y) x)) (((a a) a) a))


------------------>
index 1, term: ((λx.((y y) x)) (((a a) a) a))
obj_term: (λx.((y y) x))
body obj_term vars: {82: {'free': 2, 'bound': 0}, 83: {'free': 1, 'bound': 0}}
subj_term: (((x x) x) x)


 {1: [1, 'NORM-REDEX']}


In [25]:
def norm_term_2_v4():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, multi_app_term(x_, x_, x_, x_))
    subj_term = y_

    return App(obj_term, subj_term)


testing_procedure(norm_term_2_v4())

term: ((λx.(((x x) x) x)) y)


------------------>
index 1, term: ((λx.(((x x) x) x)) y)
obj_term: (λx.(((x x) x) x))
body obj_term vars: {84: {'free': 4, 'bound': 0}}
subj_term: x


 {1: [1, 'NORM-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.(((x x) x) x)) y)


------------------>
index 1, term: ((λx.(((x x) x) x)) y)
obj_term: (λx.(((x x) x) x))
body obj_term vars: {87: {'free': 4, 'bound': 0}}
subj_term: x


 {1: [1, 'NORM-REDEX']}


In [27]:
def norm_term_2_v5():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, multi_app_term(x_, x_, x_, x_))
    subj_term = App(multi_app_term(y_, y_, y_, y_), multi_app_term(y_, y_, y_, y_))

    return App(obj_term, subj_term)


testing_procedure(norm_term_2_v5())

term: ((λx.(((x x) x) x)) ((((y y) y) y) (((y y) y) y)))


------------------>
index 1, term: ((λx.(((x x) x) x)) ((((y y) y) y) (((y y) y) y)))
obj_term: (λx.(((x x) x) x))
body obj_term vars: {92: {'free': 4, 'bound': 0}}
subj_term: ((((x x) x) x) (((x x) x) x))


 {1: [1, 'NORM-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.(((x x) x) x)) ((((y y) y) y) (((y y) y) y)))


------------------>
index 1, term: ((λx.(((x x) x) x)) ((((y y) y) y) (((y y) y) y)))
obj_term: (λx.(((x x) x) x))
body obj_term vars: {95: {'free': 4, 'bound': 0}}
subj_term: ((((x x) x) x) (((x x) x) x))


 {1: [1, 'NORM-REDEX']}


In [28]:
def norm_term_3_v0():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, x_)
    subj_term = y_

    return App(obj_term, subj_term)


testing_procedure(norm_term_3_v0())

term: ((λx.x) y)


------------------>
index 1, term: ((λx.x) y)
obj_term: (λx.x)
body obj_term vars: {96: {'free': 1, 'bound': 0}}
subj_term: x


 {1: [1, 'NORM-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.x) y)


------------------>
index 1, term: ((λx.x) y)
obj_term: (λx.x)
body obj_term vars: {99: {'free': 1, 'bound': 0}}
subj_term: x


 {1: [1, 'NORM-REDEX']}


In [29]:
def norm_term_3_v2():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, x_)
    subj_term = Lambda(z, multi_app_term(y_, z_, x_))

    return App(obj_term, subj_term)


testing_procedure(norm_term_3_v2())

term: ((λx.x) (λy.((a y) x)))


------------------>
index 1, term: ((λx.x) (λy.((a y) x)))
obj_term: (λx.x)
body obj_term vars: {100: {'free': 1, 'bound': 0}}
subj_term: (λx.((y x) a))


 {1: [1, 'NORM-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.x) (λy.((a y) b)))


------------------>
index 1, term: ((λx.x) (λy.((a y) b)))
obj_term: (λx.x)
body obj_term vars: {103: {'free': 1, 'bound': 0}}
subj_term: (λx.((y x) a))


 {1: [1, 'NORM-REDEX']}


In [32]:
def norm_term_3_v3():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, x_)
    subj_term = Lambda(z, multi_app_term(z_, y_, z_))

    return App(obj_term, subj_term)


testing_procedure(norm_term_3_v3())

term: ((λx.x) (λy.((y a) y)))


------------------>
index 1, term: ((λx.x) (λy.((y a) y)))
obj_term: (λx.x)
body obj_term vars: {113: {'free': 1, 'bound': 0}}
subj_term: (λx.((x y) x))


 {1: [1, 'NORM-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.x) (λy.((y a) y)))


------------------>
index 1, term: ((λx.x) (λy.((y a) y)))
obj_term: (λx.x)
body obj_term vars: {116: {'free': 1, 'bound': 0}}
subj_term: (λx.((x y) x))


 {1: [1, 'NORM-REDEX']}


## * MULT TERM ***************************************************************************

In [33]:
def mult_term_1():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, App(x_, x_))
    subj_term = App(
        App(Lambda(x, App(x_, x_)), Lambda(y, Lambda(z, multi_app_term(y_, z_, x_)))),
        App(Lambda(z, App(z_, z_)), Lambda(y, Lambda(z, multi_app_term(y_, z_, x_))))
    )

    return App(obj_term, subj_term)


testing_procedure(mult_term_1())

term: ((λx.(x x)) (((λx.(x x)) (λy.(λa.((y a) x)))) ((λa.(a a)) (λy.(λa.((y a) x))))))


------------------>
index 1, term: ((λx.(x x)) (((λx.(x x)) (λy.(λa.((y a) x)))) ((λa.(a a)) (λy.(λa.((y a) x))))))
obj_term: (λx.(x x))
body obj_term vars: {118: {'free': 2, 'bound': 0}}
subj_term: (((λx.(x x)) (λy.(λa.((y a) x)))) ((λa.(a a)) (λy.(λa.((y a) x)))))


------------------>
index 7, term: ((λx.(x x)) (λy.(λa.((y a) x))))
obj_term: (λx.(x x))
body obj_term vars: {118: {'free': 2, 'bound': 0}}
subj_term: (λx.(λy.((x y) a)))


------------------>
index 19, term: ((λx.(x x)) (λy.(λx.((y x) a))))
obj_term: (λx.(x x))
body obj_term vars: {120: {'free': 2, 'bound': 0}}
subj_term: (λx.(λy.((x y) a)))


 {1: [1, 'MULT-REDEX'], 7: [3, 'MULT-REDEX'], 19: [3, 'MULT-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.(x x)) (((λy.(y y)) (λa.(λb.((a b) c)))) ((λd.(d d)) (λe.(λj.((e j) c))))))


------------------>
index 1, term: ((λx.(x x)) (((λy.(y y)) (λa.(λb.((a b) c)))) 

In [35]:
def mult_term_2_v0():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, App(x_, Lambda(y, y_)))
    subj_term = Lambda(z, multi_app_term(z_, y_, z_))

    return App(obj_term, subj_term)


testing_procedure(mult_term_2_v0())

term: ((λx.(x (λy.y))) (λa.((a y) a)))


------------------>
index 1, term: ((λx.(x (λy.y))) (λa.((a y) a)))
obj_term: (λx.(x (λy.y)))
body obj_term vars: {131: {'free': 1, 'bound': 0}, 132: {'free': 0, 'bound': 1}}
subj_term: (λx.((x y) x))


 {1: [1, 'MULT-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.(x (λy.y))) (λa.((a b) a)))


------------------>
index 1, term: ((λx.(x (λy.y))) (λa.((a b) a)))
obj_term: (λx.(x (λy.y)))
body obj_term vars: {134: {'free': 1, 'bound': 0}, 135: {'free': 0, 'bound': 1}}
subj_term: (λx.((x y) x))


 {1: [1, 'MULT-REDEX']}


In [36]:
def mult_term_2_v1():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, App(Lambda(y, y_), x_))
    subj_term = Lambda(z, multi_app_term(z_, y_, z_))

    return App(obj_term, subj_term)


testing_procedure(mult_term_2_v1())

term: ((λx.((λy.y) x)) (λa.((a y) a)))


------------------>
index 1, term: ((λx.((λy.y) x)) (λa.((a y) a)))
obj_term: (λx.((λy.y) x))
body obj_term vars: {138: {'free': 0, 'bound': 1}, 137: {'free': 1, 'bound': 0}}
subj_term: (λx.((x y) x))


------------------>
index 3, term: ((λx.x) y)
obj_term: (λx.x)
body obj_term vars: {138: {'free': 1, 'bound': 0}}
subj_term: x


 {1: [1, 'MULT-REDEX'], 3: [3, 'NORM-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.((λy.y) x)) (λa.((a b) a)))


------------------>
index 1, term: ((λx.((λy.y) x)) (λa.((a b) a)))
obj_term: (λx.((λy.y) x))
body obj_term vars: {141: {'free': 0, 'bound': 1}, 140: {'free': 1, 'bound': 0}}
subj_term: (λx.((x y) x))


------------------>
index 3, term: ((λx.x) y)
obj_term: (λx.x)
body obj_term vars: {141: {'free': 1, 'bound': 0}}
subj_term: x


 {1: [1, 'MULT-REDEX'], 3: [3, 'NORM-REDEX']}


In [37]:
def mult_term_2_v2():
    x, y, z = Var(), Var(), Var()
    x_, y_, z_ = Atom(x), Atom(y), Atom(z)

    obj_term = Lambda(x, multi_app_term(x_, x_, x_, x_))
    subj_term = Lambda(z, multi_app_term(z_, y_, z_))

    return App(obj_term, subj_term)


testing_procedure(mult_term_2_v2())

term: ((λx.(((x x) x) x)) (λy.((y a) y)))


------------------>
index 1, term: ((λx.(((x x) x) x)) (λy.((y a) y)))
obj_term: (λx.(((x x) x) x))
body obj_term vars: {143: {'free': 4, 'bound': 0}}
subj_term: (λx.((x y) x))


 {1: [1, 'MULT-REDEX']}


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
UPDATE BOUND VARS
term: ((λx.(((x x) x) x)) (λy.((y a) y)))


------------------>
index 1, term: ((λx.(((x x) x) x)) (λy.((y a) y)))
obj_term: (λx.(((x x) x) x))
body obj_term vars: {146: {'free': 4, 'bound': 0}}
subj_term: (λx.((x y) x))


 {1: [1, 'MULT-REDEX']}
